<a href="https://colab.research.google.com/github/riponalmamun/Car-Speed-and-Counting/blob/main/Car_Speed_and_Counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO
import time
from google.colab.patches import cv2_imshow # Import cv2_imshow

# load model
model = YOLO("yolov8m.pt")

# get names from classes
Classes = model.model.names

# defined Target Classes
lst = ["car", "bus", "truck"]
TargetClasses = {key: Classes[key] for key in Classes if Classes[key] in lst}
ClsInx = list(TargetClasses.keys())

# defind 2 lines to calculate speed
line1_y = 250  # first line
line2_y = 400  # second line
speed_limit = 60  # speed limit

# load video
cap = cv2.VideoCapture(r"/content/drive/MyDrive/Car.mp4")

# store dataset vechicles
TotalCounter = set()
car_dict = {}

while True :
    ret, frame = cap.read()
    if not ret:
        print("video end")
        break

    # draw lines
    cv2.line(frame, (0, line1_y), (frame.shape[1], line1_y), (255, 0, 0), 2)  ## draw first line
    cv2.line(frame, (0, line2_y), (frame.shape[1], line2_y), (0, 0, 255), 2)  ## draw second line

    # operate yolo on frame to track object
    results = model.track(frame, persist=True, classes=ClsInx, show=False, conf=0.1)

    for result in results:
        boxes = result.boxes.xyxy
        ids = result.boxes.id
        classes = result.boxes.cls

        for box, id, cls in zip(boxes, ids, classes):
            x, y, w, h = map(int, box)
            cx, cy = (x + w) // 2, (y + h) // 2  # center of car

            if int(cls) in ClsInx:
                # add car if not added before
                if int(id) not in car_dict:
                    car_dict[int(id)] = {'entered': False, 'passed': False, 'time': None, 'speed': None}

                # record pass time for first one for car
                if cy >= line1_y and not car_dict[int(id)]['entered']:
                    car_dict[int(id)]['time'] = time.time()
                    car_dict[int(id)]['entered'] = True  # passed first one

                # calc speed after pass second line after passing first line
                if car_dict[int(id)]['entered'] and not car_dict[int(id)]['passed'] and cy >= line2_y:
                    elapsed_time = time.time() - car_dict[int(id)]['time']  ## calc time between 2 lines
                    speed = 150 / elapsed_time  # speed by pexel
                    car_dict[int(id)]['speed'] = speed
                    car_dict[int(id)]['passed'] = True  # record that car passed

                    ## add car to counter for recording it
                    TotalCounter.add(int(id))

                # draw rectangle around car
                cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)

                # display ID for car
                cv2.putText(frame,
                            f"ID: {int(id)}",
                            (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.6,
                            (0, 255, 255),
                            2)

                # display speed for px/s
                if car_dict[int(id)]['speed'] is not None:
                    speed_text = f"Speed: {int(car_dict[int(id)]['speed'])} px/s"
                    color = (0, 255, 0) if car_dict[int(id)]['speed'] <= speed_limit else (0, 0, 255)  # red if car is heiest than speed limit
                    cv2.putText(frame, speed_text, (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                    # Warning if Speed higher than speed limit
                    if car_dict[int(id)]['speed'] > speed_limit:
                        cv2.putText(frame, "OVERSPEED!", (x, y - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # display the Total Vechicles if passed 2 lines only
    cv2.putText(frame, f"Total Vehicles: {len(TotalCounter)}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

    # display frame using cv2_imshow
    cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow

    if cv2.waitKey(50) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()